In [8]:
import numpy as np
import cv2


from midi_scanner.utils.ImageProcessor import ImageProcessor

# FF
# filename = "media\FFX.mp4"
# clean_frame = 0
# keyboard_roi = (0, 282, 640, 355)

# Aristochats
# filename = "media\Les Aristochats - Gammes et arpèges (piano facile).mp4"
# clean_frame = 150
# keyboard_roi = (0, 246, 635, 350)

# Au Clair de la lune
filename = "media\Au Clair De La Lune - Super Easy Piano Tutorial.mp4"
clean_frame = 0
keyboard_roi = (0, 241, 647, 358)

cap = cv2.VideoCapture(filename)
n_clusters = 5

max_frame = 500

nb_frame = 300
cap.set(cv2.CAP_PROP_POS_FRAMES,clean_frame)
image_processor = ImageProcessor(keyboard_roi)

data = None

status, image = cap.read()
image_base_roi = image_processor.get_keyboard_image(image)


In [2]:
def display_image(title, img):
    cv2.imshow(title,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
import math
def display_lines_custom(win_name, img, lines, destroy=True):
	new_img = img.copy()
	for line in lines:
		rho = int(line[0][0])
		theta = float(line[0][1])
		# if theta == 0:
		# 	new_img = cv2.line(new_img, (rho, 0), (rho, img.shape[0]), 255, 3)
		# elif theta == np.pi/2:
		# 	new_img = cv2.line(new_img, (rho, 0), (rho, img.shape[0]), (0, 255, 0), 3)
		# else:
		a = np.cos(theta)
		b = np.sin(theta)
	
		# x0 stores the value rcos(theta)
		x0 = a*rho
	
		# y0 stores the value rsin(theta)
		y0 = b*rho
	
		# x1 stores the rounded off value of (rcos(theta)-1000sin(theta))
		x1 = int(x0 + 1000*(-b))
	
		# y1 stores the rounded off value of (rsin(theta)+1000cos(theta))
		y1 = int(y0 + 1000*(a))
	
		# x2 stores the rounded off value of (rcos(theta)+1000sin(theta))
		x2 = int(x0 - 1000*(-b))
	
		# y2 stores the rounded off value of (rsin(theta)-1000cos(theta))
		y2 = int(y0 - 1000*(a))
	
		# cv2.line draws a line in img from the point(x1,y1) to (x2,y2).
		# (0,0,255) denotes the colour of the line to be
		# drawn. In this case, it is red.
		if theta == 0:
			color = (0,0,255)
			#new_img = cv2.line(new_img, (x1, y1), (x2, y2), color, 2)
			continue
		else:
			if round(theta,3) == round(np.pi/2, 3):
				color = (0,255,0)
				new_img = cv2.line(new_img, (x1, y1), (x2, y2), color, 2)
			#elif  abs(x1 -x2) < 20:
			#	cpl
			#	new_img = cv2.line(new_img, (x1, y1), (x2, y2), color, 2)
			else:
				color = (255,0,0)
					
	if destroy :
		display_image(win_name, new_img)
	else:
		cv2.imshow(win_name, new_img)

In [9]:

while True:
    
    status, image = cap.read()
    # image_base_roi = image_processor.get_keyboard_image(image)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    canny_binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    canny_binary = cv2.Canny(canny_binary, threshold1=120, threshold2=200)

    cv2.imshow("canny", canny_binary)

    # Get lines with theta = 0 (vertical lines)
    lines_canny = cv2.HoughLines(canny_binary, rho=1, theta=np.pi/24 , threshold=(image.shape[1]//5), min_theta=0, max_theta= np.pi )
    # get horizontal lines
    display_lines_custom("lines", image, lines_canny , False)

    # if cv2.waitKey(25) & 0xFF == ord('q'):
    key = cv2.waitKey(0) & 0xFF
    if key == ord('q'):
        cv2.destroyAllWindows()
        break



In [ ]:
gray = cv2.cvtColor(image_base_roi, cv2.COLOR_BGR2GRAY)

canny_binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
canny_binary = cv2.Canny(canny_binary, threshold1=120, threshold2=200)

display_image("canary", canny_binary)
# Get lines with theta = 0 (vertical lines)
lines_canny = cv2.HoughLines(canny_binary, rho=1, theta=np.pi/24 , threshold=(image_base_roi.shape[1]//5), min_theta=0, max_theta= np.pi )
# get horizontal lines
lines_height = [line[0][0] for line in lines_canny if round(float(line[0][1]), 3) == round(np.pi/2,3)]
lines_height.sort()
print(lines_height)
display_lines_custom("lines", image_base_roi, lines_canny )



In [67]:
class ResizableRectangle:
    def __init__(self, canvas, x1, y1, x2, y2):
        self.canvas = canvas
        self.rectangle = [
            canvas.create_line(x1, y1, x2, y1, fill='blue', width=2, tags='edge'),  # top
            canvas.create_line(x2, y1, x2, y2, fill='blue', width=2, tags='edge'),  # right
            canvas.create_line(x2, y2, x1, y2, fill='blue', width=2, tags='edge'),  # bottom
            canvas.create_line(x1, y2, x1, y1, fill='blue', width=2, tags='edge')   # left
        ]
        self.handles = []
        self.create_handles()
        self.bind_events()

    def create_handles(self):
        coords = [self.canvas.coords(line) for line in self.rectangle]
        self.handles = [
            self.canvas.create_oval(coords[0][0]-5, coords[0][1]-5, coords[0][0]+5, coords[0][1]+5, fill='blue', tags='handle'),  # top-left
            self.canvas.create_oval(coords[1][0]-5, coords[1][1]-5, coords[1][0]+5, coords[1][1]+5, fill='blue', tags='handle'),  # top-right
            self.canvas.create_oval(coords[2][0]-5, coords[2][1]-5, coords[2][0]+5, coords[2][1]+5, fill='blue', tags='handle'),  # bottom-right
            self.canvas.create_oval(coords[3][0]-5, coords[3][1]-5, coords[3][0]+5, coords[3][1]+5, fill='blue', tags='handle')   # bottom-left
        ]

    def bind_events(self):
        for handle in self.handles:
            self.canvas.tag_bind(handle, "<Button-1>", self.start_resize)
            self.canvas.tag_bind(handle, "<B1-Motion>", self.resize_rectangle)

    def start_resize(self, event):
        self.handle = self.canvas.find_closest(event.x, event.y)
        self.start_x = event.x
        self.start_y = event.y

    def resize_rectangle(self, event):
        handle_index = self.handles.index(self.handle)
        x1, y1, x2, y2 = self.get_current_coords()

        if handle_index == 0:  # top-left
            x1, y1 = event.x, event.y
        elif handle_index == 1:  # top-right
            x2, y1 = event.x, event.y
        elif handle_index == 2:  # bottom-right
            x2, y2 = event.x, event.y
        elif handle_index == 3:  # bottom-left
            x1, y2 = event.x, event.y

        self.update_lines(x1, y1, x2, y2)
        self.update_handles()

    def get_current_coords(self):
        coords = [self.canvas.coords(line) for line in self.rectangle]
        x1, y1 = coords[0][0], coords[0][1]
        x2, y2 = coords[2][0], coords[2][1]
        return x1, y1, x2, y2

    def update_lines(self, x1, y1, x2, y2):
        self.canvas.coords(self.rectangle[0], x1, y1, x2, y1)  # top
        self.canvas.coords(self.rectangle[1], x2, y1, x2, y2)  # right
        self.canvas.coords(self.rectangle[2], x2, y2, x1, y2)  # bottom
        self.canvas.coords(self.rectangle[3], x1, y2, x1, y1)  # left

    def update_handles(self):
        coords = [self.canvas.coords(line) for line in self.rectangle]
        handle_coords = [
            (coords[0][0], coords[0][1]),  # top-left
            (coords[1][0], coords[1][1]),  # top-right
            (coords[2][0], coords[2][1]),  # bottom-right
            (coords[3][0], coords[3][1])   # bottom-left
        ]
        for handle, (x, y) in zip(self.handles, handle_coords):
            self.canvas.coords(handle, x-5, y-5, x+5, y+5)



prop 1 0.0
prop 2 36.5
prop 3 48.66666666666667
prop 4 54.75
prop 5 58.4
prop 6 60.833333333333336
prop 7 62.57142857142857


In [68]:
height = [height for height in lines_height if ((height > (image_base_roi.shape[0] - image_base_roi.shape[0]/ 2)))][0]
height

50.0

In [ ]:
a = math.cos(theta)
 b = math.sin(theta)
 x0 = a * rho
 y0 = b * rho
 pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
 pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))

In [4]:
image_base_roi.shape

(104, 635, 3)

In [ ]:
im_height = clean_frame.shape[0]
im_bottom = clean_frame[int(im_height - (im_height / 3)):im_height, :]

gray = cv2.cvtColor(im_bottom, cv2.COLOR_BGR2GRAY)

canny_binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
canny_binary = cv2.Canny(canny_binary, threshold1=120, threshold2=200)
cv2.imshow("canny", canny_binary)
# Get lines with theta = 0 (vertical lines)
lines_canny = cv2.HoughLines(canny_binary, rho=1, theta=np.pi / 180, threshold=20, min_theta=0, max_theta=np.pi / 180)

display_lines("White key detection: clines from houghlines", clean_frame, lines_canny)

white_notes = _get_keys_from_lines(lines_canny, start_key, im_bottom.shape[1])

white_notes_img = put_white_notes_on_image(base_image=clean_frame.copy(), notes=white_notes)

cv2.imshow("notes", white_notes_img)
cv2.waitKey(0)
return white_notes

In [3]:
cv2.imshow("base_image", image_base_roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
image_base_roi.shape

(73, 640, 3)

In [10]:
import tkinter as tk

from tkinter import messagebox
from PIL import Image, ImageTk
from midi_scanner.utils.gui_utils import cv2_to_tkinter_image
top = tk.Tk()


img_tk = cv2_to_tkinter_image(image_base_roi)
print(img_tk)
# img_label = tk.Label(top, image= img_tk)
# img_label.image = img_tk
# img_label.pack()

canvas = tk.Canvas(top, width=img_tk.width(), height=img_tk.height())
canvas.pack()

# Add the image to the canvas
canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)

def move_line(event):
    canvas.moveto(line_id, 0, event.y)


# Draw the initial horizontal line
line_id = canvas.create_line(0, img_tk.height() // 2, img_tk.width(), img_tk.height() // 2, fill="red", width=2)
canvas.tag_bind(line_id, "<Button-1>", lambda event: canvas.bind("<B1-Motion>", move_line))
canvas.tag_bind(line_id, "<ButtonRelease-1>", lambda event: canvas.unbind("<B1-Motion>"))
# C = Canvas(top, bg="blue", height=250, width=300)
# coord = 10, 50, 240, 210
# photo = ImageTk.PhotoImage()
# img = C.create_image(0,0, image= photo), 
# arc = C.create_arc(coord, start=0, extent=150, fill="red")
# line = C.create_line(10,10,200,200,fill='white')
# C.pack()
top.mainloop()

pyimage6


In [3]:
import tkinter as tk
from PIL import Image, ImageTk

class ResizableRectangle:
    def __init__(self, canvas, x1, y1, x2, y2):
        self.canvas = canvas
        self.rectangle = [
            canvas.create_line(x1, y1, x2, y1, fill='blue', width=2, tags='edge'),  # top
            canvas.create_line(x2, y1, x2, y2, fill='blue', width=2, tags='edge'),  # right
            canvas.create_line(x2, y2, x1, y2, fill='blue', width=2, tags='edge'),  # bottom
            canvas.create_line(x1, y2, x1, y1, fill='blue', width=2, tags='edge')   # left
        ]
        self.handles = []
        self.create_handles()
        self.bind_events()

    def create_handles(self):
        coords = [self.canvas.coords(line) for line in self.rectangle]
        self.handles = [
            self.canvas.create_oval(coords[0][0]-5, coords[0][1]-5, coords[0][0]+5, coords[0][1]+5, fill='blue', tags='handle'),  # top-left
            self.canvas.create_oval(coords[1][0]-5, coords[1][1]-5, coords[1][0]+5, coords[1][1]+5, fill='blue', tags='handle'),  # top-right
            self.canvas.create_oval(coords[2][0]-5, coords[2][1]-5, coords[2][0]+5, coords[2][1]+5, fill='blue', tags='handle'),  # bottom-right
            self.canvas.create_oval(coords[3][0]-5, coords[3][1]-5, coords[3][0]+5, coords[3][1]+5, fill='blue', tags='handle')   # bottom-left
        ]

    def bind_events(self):
        for handle in self.handles:
            self.canvas.tag_bind(handle, "<Button-1>", self.start_resize)
            self.canvas.tag_bind(handle, "<B1-Motion>", self.resize_rectangle)

    def start_resize(self, event):
        
        print(self.canvas.find_withtag("current")[0])
        self.handle = self.canvas.find_closest(event.x, event.y)
        self.start_x = event.x
        self.start_y = event.y

    def resize_rectangle(self, event):
        print("resize")
        handle_index = self.handles.index(self.handle)
        x1, y1, x2, y2 = self.get_current_coords()

        if handle_index == 0:  # top-left
            x1, y1 = event.x, event.y
        elif handle_index == 1:  # top-right
            x2, y1 = event.x, event.y
        elif handle_index == 2:  # bottom-right
            x2, y2 = event.x, event.y
        elif handle_index == 3:  # bottom-left
            x1, y2 = event.x, event.y

        self.update_lines(x1, y1, x2, y2)
        self.update_handles()

    def get_current_coords(self):
        coords = [self.canvas.coords(line) for line in self.rectangle]
        x1, y1 = coords[0][0], coords[0][1]
        x2, y2 = coords[2][0], coords[2][1]
        return x1, y1, x2, y2

    def update_lines(self, x1, y1, x2, y2):
        self.canvas.coords(self.rectangle[0], x1, y1, x2, y1)  # top
        self.canvas.coords(self.rectangle[1], x2, y1, x2, y2)  # right
        self.canvas.coords(self.rectangle[2], x2, y2, x1, y2)  # bottom
        self.canvas.coords(self.rectangle[3], x1, y2, x1, y1)  # left

    def update_handles(self):
        coords = [self.canvas.coords(line) for line in self.rectangle]
        handle_coords = [
            (coords[0][0], coords[0][1]),  # top-left
            (coords[1][0], coords[1][1]),  # top-right
            (coords[2][0], coords[2][1]),  # bottom-right
            (coords[3][0], coords[3][1])   # bottom-left
        ]
        for handle, (x, y) in zip(self.handles, handle_coords):
            self.canvas.coords(handle, x-5, y-5, x+5, y+5)

def main():
    # Create the main window
    root = tk.Tk()
    root.title("Resizable Rectangle on Image")

    # Load the image
    image_path = "./media/test_frame.png"  # Replace with your image path
    image = Image.open(image_path)
    photo = ImageTk.PhotoImage(image)

    # Create a canvas widget
    canvas = tk.Canvas(root, width=photo.width(), height=photo.height())
    canvas.pack()

    # Add the image to the canvas
    canvas.create_image(0, 0, anchor=tk.NW, image=photo)
    canvas.img = photo

    # Draw a resizable rectangle
    resizable_rect = ResizableRectangle(canvas, 50, 50, 150, 150)

    # Start the Tkinter event loop
    root.mainloop()

if __name__ == "__main__":
    main()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\simon\AppData\Local\Temp\ipykernel_48080\937122197.py", line 33, in start_resize
    print(self.find_withtag("current")[0])
AttributeError: 'ResizableRectangle' object has no attribute 'find_withtag'
